In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse("dataset/training_set.xml")
root = tree.getroot()

In [3]:
def xmlToDict(root):
    def processChild(child):
        text = child[0].text
        aspectsTree = child[1]
        aspects = {aspect.attrib['category'] : aspect.attrib['polarity']
                    for aspect in aspectsTree}
        return dict({'text' : text}, **aspects)
    return {child.attrib['rid'] : processChild(child) for child in root}

In [4]:
def dictToAspectDf(data):
    def generateLabel(listAspects):
        aspects = ['FOOD', 'AMBIANCE', 'PRICE', 'SERVICE']
        return {aspect : (aspect in listAspects) for aspect in aspects}
    newData = {}
    for k, v in data.items():
        newData[k] = {'text' : v['text'], **generateLabel(v.keys())}
    return pd.DataFrame.from_dict(newData, orient='index')
        
def dictToSentimentDf(data):
    aspects = ['FOOD', 'AMBIANCE', 'PRICE', 'SERVICE']
    def generateLabel(dictAspects):
        listAspects = dictAspects.keys()
        return {aspect : (dictAspects[aspect] if aspect in listAspects else 'UNKNOWN') for aspect in aspects}
    newData = {}
    for k, v in data.items():
        newData[k] = {'text' : v['text'], **generateLabel(v)}
    return pd.DataFrame.from_dict(newData, orient='index')

In [5]:
data = xmlToDict(root)

In [6]:
dictToSentimentDf(data).to_csv('dataset/preProcessSentiment.csv')
dictToAspectDf(data).to_csv('dataset/preProcessAspect.csv')